In [1]:
# from limes_x.agents import Ambassador
import os, sys
from pathlib import Path
from typing import Iterable
import yaml
from sshtunnel import SSHTunnelForwarder
import requests
from typing import Any
import pickle, gzip, base64

from limes_x.models import Namespace, DataInstance, Endpoint, Transform, Solve, KeyGenerator
from limes_x.compute_module import ComputeModule
from limes_x.outpost.models import Message, Context
from limes_x.outpost.config import HOST, PORT, VER

In [2]:
class Connection:
    def __init__(self, host: str) -> None:
        self.server = SSHTunnelForwarder(
            host,
            remote_bind_address=(HOST, PORT)
        )

        self.server.start()
        self._started = True

        self._kg = KeyGenerator(True)
        self._KEY_LEN = 12
        self.compression_level = 3

        code, _ = self.Ping()
        assert code == 200, f"failed to connect to outpost [{host}]"

    def _pack(self, data: Any):
        return base64.urlsafe_b64encode(
            gzip.compress(
                pickle.dumps(data, protocol=pickle.HIGHEST_PROTOCOL),
                compresslevel=self.compression_level
            ),
        ).decode("ascii")
    
    def _unpack(self, raw: str):
        return pickle.loads(gzip.decompress(base64.urlsafe_b64decode(raw)))
    
    def _send(self, context: Context, payload: Any=None):
        res = requests.post(
            f"http://localhost:{self.server.local_bind_port}/{VER}",
            json=Message(
                key=self._kg.GenerateUID(self._KEY_LEN),
                context=context,
                payload=self._pack(payload),
            ).ToDict(),
        )

        msg = Message.FromDict(res.json())
        msg.payload = self._unpack(msg.payload)
        return res.status_code, msg

    def Ping(self, msg:str="hello"):
        return self._send(
            Context.PING,
            msg,
        )
    
    def SetHome(self, path: Path|str):
        return self._send(
            Context.SET_HOME,
            Path(path),
        )
    
    def ReloadModules(self):
        return self._send(Context.RELOAD_MODULES)
    
    def ListTransforms(self):
        return self._send(Context.LIST_TRANSFORMS)

    def Start(self):
        if not self._started: self.server.start()

    def Stop(self):
        self.server.stop()
        self._started = False

class Config:
    def __init__(self, path: Path) -> None:
        with open(path) as y:
            config = yaml.safe_load(y)

        self.connections: list[Connection] = []
        for host in config.get("outposts", {}).get("ssh", []):
            self.connections.append(Connection(host))

class Ambassador:
    def __init__(self, home: Path|str) -> None:
        home = Path(home)
        self.home = home
        if not home.exists():
            os.makedirs(home, exist_ok=True)
        else:
            assert home.is_dir(), f"home path [{home}] isn't folder"

        self.config = Config(home.joinpath("config.yml"))
        self.connections: list[Connection] = self.config.connections

    def __enter__(self):
        for con in self.connections:
            con.Start()
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        for con in self.connections:
            con.Stop()

    def ListTransforms(self):
        for con in self.connections:
            x = con.ListTransforms()
            print(x)

    def GetPlan(self, given: Iterable[DataInstance], target: Transform):
        # given_endpoints = [i.endpoint for i in given]
        # plan = Solve(given_endpoints, target, [m.transform for m in self.modules])
        # return plan
        return

    def Execute(self, plan):
        pass

ns = Namespace()
a_proto = Endpoint(ns, {"a"})
a = DataInstance(a_proto, "/home/tony/workspace/python/Limes-all/Limes-x/test/outposts/local_outpost/workspace/a.txt")

target = Transform(ns)
target.AddRequirement({"b"})

with Ambassador("./test_ws") as lx:
    lx.ListTransforms()
# plan = lx.GetPlan([a], target)
# print(plan)
# lx.Execute(plan)

(200, Message(key='g9KnFDw_qCkq', context=<Context.RESPONSE: 4>, payload=[]))
